# 透過 Python 擷取網頁上的資料

在大數據變成顯學的時代，今天我們要做資料分析，若沒有大量的資料，是無法分析出任何有價值的資訊出來的。

但是在一般情況下，我們都沒有大量的資料可以做分析，但幸運的是，由於網路的蓬勃發展，獲取任何一個領域的資料比起過往相對就變得容易許多。但是，由於網頁上的資料非常多，若透過手動的方式截取資料，就不是一個很有效率的方法了。

因此這時候，透過程式自動從網路上蒐集資料這個問題也變得更加重要，只需一個簡單的程式，就能透過低成本並且自動化的方式從網頁上獲得大量的資料，因此**學習與實作網頁爬蟲成為一個投資報酬率極高的事務**，而 Python 語言由於生態系龐大，套件衆多，也讓用 Python 實作爬蟲比起其他語言相對簡單許多。

## 請先開啓範例網頁：

http://pythonscraping.com/pages/warandpeace.html

*以上練習用網頁由 Ryan Mitchell 維護，可以的話，大家買一本他的書支持他繼續維護這個網站：[連結](https://www.books.com.tw/products/0010800965)

在進入網頁之後，試試看直接點擊網頁，然後右鍵 -> 儲存，接下來瀏覽器會將一個副檔名為 **html** 的檔案存下來。由此我們發現，任何一個我們看得見的網頁，其實都是一個副檔名為 **.html** 的檔案，問題是，這個 **html** 檔案是從何而來？它的原理又是什麽？

## 網頁開發基礎： HTTP 溝通協定

HTTP 用白話講，就像是**電腦與電腦之間的共同語言**，電腦需要通過這個共同語言，才能在網絡上面互相溝通。

![](https://drive.google.com/uc?export=download&id=1UiDUXriZVo83ePdiSQTLS4QWnCN_YsVu)

而一個 HTTP 的請求 (Request) 是根據 **HTTP 動詞** (HTTP Verb) 以及**網址**才能運作，舉例來説，當你在輸入 https://www.google.com/ ，連至 Google 的首頁時，整個背後的運作流程是：

1. 用戶端 (你的瀏覽器) 針對 Google 的雲端伺服器發送了一個 GET Request
2. 而 Google 的雲端伺服器在收到請求後，將需要呈現該網頁的資料都計算完成
3. 接著 Google 的雲端伺服器會回傳一個回應 (Response)，這個 Response 内通常就包含了一個 html 檔案
4. 用戶端 (你的瀏覽器) 在下載了請求回傳的 html 檔案之後，將 html 程式碼渲染成網頁，呈現給使用者

## HTTP Request 的種類

一般來説，Request 有 GET 與 POST：

- GET 代表我需要查詢 / 顯示資料，像是 GET https://www.facebook.com/ 代表查詢 facebook 首頁
- POST 代表我需要新增資料，通常用於網頁上的表單，像是 POST https://www.foodpanda.com/orders 代表新增訂單

## HTTP response 的種類

- 一般伺服器通常是回傳 html 網頁檔案
- 但若是一些功能像是下載 / 輸出報表，Response 則是一個 xlsx / csv 檔案


## 從 Excel 的角度來理解...

像是 FB, Google, Yahoo 等網站，**其實都像是一個個運行在雲端上的 Excel 函數**。

而要使用該函數時，就必須透過一個 HTTP 的請求 (Request) 來呼叫該函數。讓瀏覽器發送 HTTP Request 的方法就是輸入網址，就像是在 Excel 上輸入公式一樣。

而該公式若執行成功，不同於 Excel 是將結果顯示在工作表上，HTTP Request 的結果就是一個 Response，該 Response 通常是顯示在瀏覽器上的網頁，也有可能是 xlsx / csv 檔案


## 用 Python 實作爬蟲

Python 用來實作爬蟲的兩個主流套件：

- BeautifulSoup [官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)

- PyQuery [官方文件](https://pythonhosted.org/pyquery/)

簡單來説：

**BeautifulSoup** 是比較貼近**程式設計師**的角度去思考

**PyQuery** 貼近**網頁開發者**的角度去思考

在這堂課考量到並不是每一個人都具備網頁開發的背景，因此我們會使用 BeautifulSoup 套件來實作爬蟲

## 網頁開發 101

任何網頁都是由 **html 標籤(tag)** 所組成，基本結構如下
```html
<標籤名稱 class="類別名稱">內容</標籤名稱>
<標籤名稱 id="id名稱">內容</標籤名稱>
```
今天我們要擷取的任何內容，一定是被包裹在在某一個標籤裡面
而今天若網頁開發者需要改變任何一個標籤的**樣式**，就需要用到 **css** 語法
以上面的網頁為例，人名都是以綠色顯示，所以就先宣告一個名為 **green** 的 css 類別:

```html
<style>
.green{
	color:#55ff55;
}
</style>
```

若今天希望讓一個標籤的内容文字變成綠色，可以使用定義好的 .green 這個 css 類別：

```html
<span class="green">Prince Vasili Kuragin</span>
```

*想了解更多 html 可以看一下 Mozilla 官網的教學：[HTML 基礎](https://developer.mozilla.org/zh-TW/docs/Learn/Getting_started_with_the_web/HTML_basics)

<h1> War and Peace </h1> #h表示header
<p>...</p> #p表示paragraph

## 使用 標籤 「類別名稱」取得資料

要擷取資料前，首先需要透過方法**選擇**到該標籤
##  BeautifulSoup 套件

典故來自 Alice in WonderLand 裏面一首同名的詩，由假海龜 (Mock Turtle) 所唱，影射英國料理假海龜湯...

有興趣自己可以去 Google, 不多説了...

想查看 BeautifulSoup 套件的功能請看一下官方的中文文件：

[BeautifulSoup 官方文件](https://www.crummy.com/software/BeautifulSoup/bs4/doc.zh/)


```python
from bs4 import BeautifulSoup
import requests

# 針對網頁發送 GET Reqeust
res = requests.get("http://pythonscraping.com/pages/warandpeace.html")
# 將回傳的 Response 内的文字用 BeautifulSoup 解析
html = BeautifulSoup(res.text, "html.parser")
```

In [2]:
import bs4
import requests

res = requests.get("http://pythonscraping.com/pages/warandpeace.html")
res.text

'<html>\n<head>\n<style>\n.green{\n\tcolor:#55ff55;\n}\n.red{\n\tcolor:#ff5555;\n}\n#text{\n\twidth:50%;\n}\n</style>\n</head>\n<body>\n<h1>War and Peace</h1>\n<h2>Chapter 1</h2>\n<div id="text">\n"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don\'t tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy \'faithful slave,\' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.</span>"\n<p/>\nIt was in July, 1805, and the speaker was the well-known <span class="green">Anna\nPavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya\nFedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man\nof high rank

## 另外再介紹一下 find() 方法...

```python
from bs4 import BeautifulSoup
import requests

res = requests.get("http://pythonscraping.com/pages/warandpeace.html")
html = BeautifulSoup(res.text, 'html.parser')
# 我們把該網頁使用 'green' css 類別的 span 標籤過濾出來...
name = html.find("span", {"class": "green"})
print(name)
```

In [ ]:
#html控制網頁的排版  css控制樣式（呈現方式）

In [5]:
html = bs4.BeautifulSoup(res.text, "html.parser")
html

<html>
<head>
<style>
.green{
	color:#55ff55;
}
.red{
	color:#ff5555;
}
#text{
	width:50%;
}
</style>
</head>
<body>
<h1>War and Peace</h1>
<h2>Chapter 1</h2>
<div id="text">
"<span class="red">Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.</span>"
<p></p>
It was in July, 1805, and the speaker was the well-known <span class="green">Anna
Pavlovna Scherer</span>, maid of honor and favorite of the <span class="green">Empress Marya
Fedorovna</span>. With these words she greeted <span class="green">Prince Vasili Kuragin</span>, a man
of high rank and importance, who was the firs

In [34]:
span_list = html.findAll("span") #findAll會把資料裝進list
span_list

[<span>首頁</span>,
 <span>更多</span>,
 <span>台股代號查詢</span>,
 <span id="marketTimeCountDown">距離美股開盤還有3小時41分鐘</span>,
 <span class="more"><a data-ylk="slk:more;itc:0;sec:symbol-news;elm:itm;elmt:link" href="https://tw.stock.yahoo.com/q/h?s=2330">» 更多</a></span>,
 <span class="meta">(鉅亨網 2020/08/31 15:36)</span>,
 <span class="meta">(時報資訊 2020/08/31 12:15)</span>,
 <span class="meta">(中央社 2020/08/31 11:05)</span>,
 <span class="meta">(時報資訊 2020/08/31 10:35)</span>]

In [16]:
span_list = html.findAll("span", { "class" : "green" })
span_list

[<span class="green">Anna
 Pavlovna Scherer</span>, <span class="green">Empress Marya
 Fedorovna</span>, <span class="green">Prince Vasili Kuragin</span>, <span class="green">Anna Pavlovna</span>, <span class="green">St. Petersburg</span>, <span class="green">the prince</span>, <span class="green">Anna Pavlovna</span>, <span class="green">Anna Pavlovna</span>, <span class="green">the prince</span>, <span class="green">the prince</span>, <span class="green">the prince</span>, <span class="green">Prince Vasili</span>, <span class="green">Anna Pavlovna</span>, <span class="green">Anna Pavlovna</span>, <span class="green">the prince</span>, <span class="green">Wintzingerode</span>, <span class="green">King of Prussia</span>, <span class="green">le Vicomte de Mortemart</span>, <span class="green">Montmorencys</span>, <span class="green">Rohans</span>, <span class="green">Abbe Morio</span>, <span class="green">the Emperor</span>, <span class="green">the prince</span>, <span class="green">Pri

In [17]:
tag = span_list[0]
tag.text

'Anna\nPavlovna Scherer'

In [18]:
for tag in span_list:
    print(tag.text)
    print("=======")

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


# 網頁爬蟲實戰：(臺股爬蟲)

我們想要截取資料的網頁：[Yahoo Stock 台積電](https://tw.stock.yahoo.com/q/q?s=2330)

![](https://drive.google.com/uc?export=download&id=1samMUBW19ooC7UsntphFGG6QKjL5337R)

網址是：

https://tw.stock.yahoo.com/q/q?s=2330

```python
# 來試試看爬 yahoo stock 的網頁...
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
# 用 html 格式解碼 爬下來的檔案
html = BeautifulSoup(res.text, 'html.parser')
print(html)
```


In [21]:
from bs4 import BeautifulSoup
import requests

res = requests.get("https://tw.stock.yahoo.com/q/q?s=2330")
res.text

'<!doctype html public "-//w3c//dtd html 4.01//en" "http://www.w3.org/tr/html4/strict.dtd">\n<html>\n<head>\n    <meta name="oath:guce:product-eu" content="false"/>\n    <meta name="oath:guce:consent-host" content="guce.yahoo.com"/>\n    <meta name="oath:guce:locale" content="zh-Hant-TW"/>\n    <meta name="oath:guce:report-only" content="false"/>\n    <meta name="oath:guce:inline-consent" content="true"/>\n    <script src="https://s.yimg.com/oa/guce.js" async></script>\n<!-- Rapid begin. -->\n\n<script language="JavaScript" src="https://s.yimg.com/ss/rapid3.js"></script>\n<script language="JavaScript" src="/__rapid-worker-1.2.js"></script> <!-- rapid should require worker file itself, this should be unnessary, but require it anyway. -->\n<script language="JavaScript">\n    YAHOO.i13n.WEBWORKER_FILE = "/__rapid-worker-1.2.js";\n    var pageParams = {"site":"finance","lang":"zh-Hant-TW","mrkt":"tw","pt":"utility","pstcat":"equities","pct":"qsp","ticker":"2330.TW"}\n    if (typeof article

In [24]:
html = BeautifulSoup(res.text, "html.parser") #html.parser將複雜字串變成html的形式
html

<!DOCTYPE doctype html public "-//w3c//dtd html 4.01//en" "http://www.w3.org/tr/html4/strict.dtd">

<html>
<head>
<meta content="false" name="oath:guce:product-eu"/>
<meta content="guce.yahoo.com" name="oath:guce:consent-host"/>
<meta content="zh-Hant-TW" name="oath:guce:locale"/>
<meta content="false" name="oath:guce:report-only"/>
<meta content="true" name="oath:guce:inline-consent"/>
<script async="" src="https://s.yimg.com/oa/guce.js"></script>
<!-- Rapid begin. -->
<script language="JavaScript" src="https://s.yimg.com/ss/rapid3.js"></script>
<script language="JavaScript" src="/__rapid-worker-1.2.js"></script> <!-- rapid should require worker file itself, this should be unnessary, but require it anyway. -->
<script language="JavaScript">
    YAHOO.i13n.WEBWORKER_FILE = "/__rapid-worker-1.2.js";
    var pageParams = {"site":"finance","lang":"zh-Hant-TW","mrkt":"tw","pt":"utility","pstcat":"equities","pct":"qsp","ticker":"2330.TW"}
    if (typeof articlePageParams === 'object') {
   

In [44]:
table = html.findAll("table", { "border" : "2"})[0]
table.findAll("tr") #搜尋table底下所有的tr標籤
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
last_close = float(tds[7].text)
last_close

435.0

In [46]:
open_price = float(tds[8].text)
high_price = float(tds[9].text)
low_price = float(tds[10].text)
close_price = float(tds[2].text)

open_price, high_price, low_price, close_price

(437.0, 439.5, 426.5, 426.5)

# 問題是找到有用的資料如同大海撈針...

---
## 分析一下我們要爬的網頁

收盤價是被封裝在一個 **table** 標籤内部的一個 **td** 標籤


---
## table 標籤

一般網頁若要呈現重要的資訊，都會將資訊以網頁表格的形式呈現

今天若要利用 **html** 實作網頁上的表格，需要名爲 **table** 的標籤

換句話説，今天網頁上我們有興趣的資料，十之八九都是被封裝在 **table** 底下

把 **table** 的結構搞懂，就成了一件重要的事情。

---
# html table 標籤的結構

網頁上的資料大多都是匯整在表格、而 html 的表格則是由 table 標籤構成的：

![](https://drive.google.com/uc?export=download&id=1dmYx5qOD21tPWHATzIm6IxDnfO96sUac)

該 **table** 標籤内有兩個 **tr** 標籤

第二個 tr 標籤内的第八個 td 標簽是我們要的收盤價

![](https://drive.google.com/uc?export=download&id=109M71HQtR-kfZMklVit7AZRpIH4zhfe_)

---
# html 標簽的關聯

簡單來説，就是一個樹狀圖的概念：

![](https://drive.google.com/uc?export=download&id=1n2yZlQtj7EK2_NNIBXvEVGeEKBogfgYZ)

---
# 延申閲讀

HTML Table 教學

w3 school： [連結](https://www.w3schools.com/html/html_tables.asp)

Mozilla：[連結](https://developer.mozilla.org/zh-TW/docs/Web/HTML/Element/table)

---

## 開始實作台股爬蟲

```python
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
# 搜尋整個網頁裡的 table 標籤，將所有的表格讀取出來
html.findAll("table")
# 我們就發現網頁内有多個 table...
```

## 檢查所有搜尋到的 table

仔細觀察一下，我們發現目標 table 的 **border** 屬性是被設定成 **2**

因此我們可以將程式碼修改爲...

```python
table = html.findAll("table", { "border": 2 })[0]
table
```

## 找尋 table 裡第二個 tr 標籤內所有的 td 標籤

```python
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
tds
```

## 讀取出昨日收盤價

```python
last_close = tds[7].text
print(f"台積電今日收盤價：${last_close}")
```

## 完成版網頁爬蟲

```python
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
table = html.findAll("table", { "border": 2 })[0]
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close = tds[7].text
print(f"台積電昨日收盤價：${last_close}")
```

## 解法二


```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 搜尋整個網頁裡，內容為 '個股資料' 的 html 標籤, 關聯到 table 最外層
table = html.findAll(text='個股資料')[0].parent.parent.parent
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
data_row = table.findAll('tr')[1].findAll('td')
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_price = data_row[7].text
print(f"台積電昨日收盤價：${last_close}")
```

In [49]:
txt = html.findAll(text = "個股資料")[0]
txt

'個股資料'

In [63]:
table = txt.parent.parent.parent
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
last_close = float(tds[7].text)
last_close 

435.0

## 解法三...

![](https://drive.google.com/uc?export=download&id=1IJp5dze4stcwni0FNs2ThIjzYM_vYp4o)

注意 **tt** 是一個獨特的 css class，而 **tt** 只有被套用在最後一個 **td** 標籤上

```python
from bs4 import BeautifulSoup
import requests

doc = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(doc.text, 'html.parser')
# 尋找 class 屬性為 tt 的 td 標籤
last_td = html.find("td", {"class": "tt"})
# find_previous_sibling('td') 代表尋找前一個 (左邊) td 標籤
last_close = last_td.find_previous_sibling('td').find_previous_sibling('td').find_previous_sibling('td').find_previous_sibling('td').text
print(f"台積電昨日收盤價：${last_close}")
```

In [67]:
last_td = html.findAll("td", {"class" : "tt"})[0]
last_td

<td align="center" class="tt" width="137">
<a href="/q/ts?s=2330">成交彙整</a><br/><a href="/q/ta?s=2330">技術</a>　<a href="/q/h?s=2330">新聞</a><a href="/d/s/company_2330.html"><br/>基本</a>　<a href="/d/s/credit_2330.html">籌碼</a><a href="https://tw.rd.yahoo.com/referurl/stock/other/SIG=125v47s73/**https://tw.screener.finance.yahoo.net/screener/check.html?symid=2330" style="color:red" target="_blank"><br/>個股健診</a></td>

In [77]:
low_price = float(last_td.find_previous_sibling("td").text)
high_price = float(last_td.find_previous_sibling("td").find_previous_sibling("td").text)
open_price = float(last_td.find_previous_sibling("td").find_previous_sibling("td").find_previous_sibling("td").text)

low_price, high_price, open_price

(426.5, 439.5, 437.0)

In [90]:
#另一個姊妹功能 .find_next_sibling()

In [89]:
first_td = html.findAll("td", { "width" : "105" })[0]
first_td.find_next_sibling("td").text

'14:30'

## 隨堂練習

請試試看從 Yahoo 股市網頁將 2330 的**開盤價、最高價、最低價**、以及**成交價**讀取出來：

```python
from bs4 import BeautifulSoup
import requests

res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
table = html.findAll("table", { "border": 2 })[0]
# 找尋 table 裡第二個 tr 標籤內所有的 td 標籤
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
# 選取該 row 第八個 td 標籤，擷取標籤內文字
last_close = tds[7].text
open_price = ________________
high_price = ________________
low_price = ________________
close_price = ________________
```

## 將結果寫入 Excel

用 xlwings 開啓 **stock_price_data.xlsx**：

```python
import xlwings as xw
import time

wb = xw.Book(r"stock_price_data.xlsx")
sheet = wb.sheets["2330"]
```

偵測最後一個 row:

```python
last_row = sheet.range("A1").end("down").row
last_row
```

In [92]:
import xlwings as xw

wb = xw.Book(r"stock_price_data.xlsx")
wb

<Book [stock_price_data.xlsx]>

In [93]:
sheet = wb.sheets["2330"]
sheet

<Sheet [stock_price_data.xlsx]2330>

In [95]:
sheet.range("B96").value = close_price

## 產生格式化的時間字串

```python
import time

time.strftime("%Y/%m/%d")
```

In [98]:
import time

time.strftime("%Y/%m/%d")

'2020/09/01'

## 將時間與收盤價寫入 Excel 

```python
sheet.range(f"A{last_row+1}").value = time.strftime("%Y/%m/%d")
sheet.range(f"B{last_row+1}").value = closing_price
```

In [114]:
last_row = sheet.range("A1").end("down").row
sheet.range(f"A{last_row + 1}").value = time.strftime("%Y/%m/%d")
sheet.range(f"B{last_row + 1}").value = close_price

# 完成版程式碼

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import xlwings as xw
# 截取台積電的資料 
res = requests.get('https://tw.stock.yahoo.com/q/q?s=2330')
html = BeautifulSoup(res.text, 'html.parser')
table = html.findAll("table", { "border": 2 })[0]
table_row = table.findAll("tr")[1]
tds = table_row.findAll("td")
closing_close = tds[2].text
# 將台積電資料寫入 Excel
wb = xw.Book(r"stock_price_data.xlsx")
date = time.strftime("%Y/%m/%d")
sheet = wb.sheets["TW2330"]
last_row = sheet.range("B1").end("down").row
sheet.range(f"B{last_row+1}").value = closing_price
sheet.range(f"A{last_row+1}").value = date

# 小結：

1. 學習與實作網頁爬蟲是一個**投資報酬率極高的事務**
2. Python 語言由於使用者衆多，**與爬蟲相關的套件、解決方案、與教學也多，讓實作變得相對簡單**
3. 實作上，最困難的部分在於**解析網頁的 html 結構**
4. 網頁的資料很大的機率都是被封裝在 **table** 這個 html 標簽下
5. 但若今天**網頁改版，原先寫好的爬蟲就有可能截取不到資料**

# 功課：匯率爬蟲

請寫一個網頁爬蟲，截取臺灣銀行牌告匯率網頁：

http://rate.bot.com.tw/xrt?Lang=zh-TW


並且將匯率資料用以下格式呈現在 Excel 内：
![](https://drive.google.com/uc?export=download&id=1YCl-QcAJCW951AhosB3HhuV7Fwy3hjMZ)